In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm
import re

#드라이버 설치
https://chromedriver.chromium.org/downloads
#드라이버랑 크롬버젼이랑 일치시켜주어야함. 드라이버 다운 받을 때 참고(메뉴 => 도움말 => Chrome 정보에서 확인)

In [2]:
#Step 1. 크롬 웹브라우저 실행
#경로설정
path = "C:/Users/bae/Desktop/chromedriver.exe"   # 맥은 "chromedriver"
driver = webdriver.Chrome(path)

In [3]:
txt_file_path = 'productUrlList.txt'

In [4]:
#productUrlList.txt에서 링크 리스트 불러오기
r = open(txt_file_path, mode='rt', encoding='utf-8')
productUrlList = r.readlines()
len(productUrlList)

633

In [59]:
import json

def toJson(data, file_path) :
    with open(file_path, 'w', encoding='utf-8') as file :
        json.dump(data, file)
    return 

txt_file_path_ls = ['productUrlList_간편식_냉장_냉동.txt', 'productUrlList_과일.txt']

for txt_file_path in txt_file_path_ls :

    #productUrlList.txt에서 링크 리스트 불러오기
    r = open(txt_file_path, mode='rt', encoding='utf-8')
    productUrlList = r.readlines()

    error_ls = []
    prod_data = []
    satisfaction_data = []

    for prod_url in tqdm(productUrlList) :
        try :
            driver.get(prod_url)
            time.sleep(1)

            #상품 정보 뽑기

            #상품 고유번호
            prod_idx = re.sub('http://www.tmon.co.kr/deal/','',prod_url)
            prod_idx = re.sub('\n','',prod_idx)
            prod_idx = int(prod_idx)

            #상품명
            productTitle= driver.find_element_by_class_name("deal_title_main").text

            #상품 카테고리 최대 6층
            category_ls = []
            for i in range(1, 7) :
                try :
                    category= driver.find_element_by_class_name("category._depth{}".format(i)).text
                    category_ls.append(category)
                except :
                    break

            #상품 가격
            productPrice = driver.find_element_by_class_name("number_unit").text #'21,900원'으로 나오기 때문에 전처리 필요 정규표현식으로 숫자만 남기고 int
            temp = re.findall('[0-9]*', productPrice)
            productPrice = int(''.join(temp))

            #구매횟수 
            buy_count = driver.find_element_by_class_name("deal_price_buy_count").text
            temp = re.findall('[0-9]*', buy_count)
            buy_count = int(''.join(temp))

            #상품 만족도 star_rate_num
            try :
                prod_satis = float(driver.find_element_by_class_name("grade_average_score").text)
            except :
                prod_satis = 0.

            #상품이미지 url
            image = driver.find_element_by_class_name("active").find_element_by_tag_name('img').get_attribute('src')

            #상품종료여부(품절 = 0, 구매가능 = 1)
            if driver.find_elements_by_class_name("col2")[1].text == '바로구매' :
                stocks = 1
            else : 
                stocks = 0


            prod_data.append({
                'prod_idx' : prod_idx,
                'image' : image,
                'title' : productTitle,
                'category' : category_ls,
                'price' : productPrice,
                'stocks' : stocks,
                'prod_satis' : prod_satis,
                'prod_url' : prod_url,
                'buy_count' : buy_count
            })

            #상품 만족도 & 배송 만족도
            customer_satisfaction = driver.find_elements_by_class_name("customer_satisfaction_box")
            avg_seller_prod_satis = int(customer_satisfaction[0].find_element_by_class_name('num').text)
            avg_seller_ship_satis = int(customer_satisfaction[1].find_element_by_class_name('num').text)

            try : #옵션 없는 경우 존재
                review_stats = driver.find_elements_by_class_name("review_stats_item")

                #옵션1과 만족도 (ex: 맛 - 73)
                use_opt1 = review_stats[0].find_element_by_class_name('keyword').text
                avg_use_satis1 = review_stats[0].find_element_by_class_name('pct').text
                avg_use_satis1 = re.sub('\%','',avg_use_satis1)
                avg_use_satis1 = int(avg_use_satis1)

                #옵션2와 만족도
                use_opt2 = review_stats[1].find_element_by_class_name('keyword').text
                avg_use_satis2 = review_stats[1].find_element_by_class_name('pct').text
                avg_use_satis2 = re.sub('\%','',avg_use_satis2)
                avg_use_satis2 = int(avg_use_satis2)

            except :
                use_opt1 =''
                avg_use_satis1 = 0
                use_opt2 =''
                avg_use_satis2 = 0

            satisfaction_data.append({
                'prod_idx' : prod_idx,
                'avg_seller_prod_satis' : avg_seller_prod_satis,
                'avg_seller_ship_satis' : avg_seller_ship_satis,
                'use_opt1' : use_opt1,
                'avg_use_satis1' : avg_use_satis1,
                'use_opt2' : use_opt2,
                'avg_use_satis2' : avg_use_satis2,
                'prod_satis' : prod_satis,
                'prod_url' : prod_url,   
            })

        except : 
            error_ls.append(prod_url)

    toJson(prod_data, 'prod_data_{}.json'.format(txt_file_path[15:-4]))
    toJson(satisfaction_data, 'satisfaction_data_{}.json'.format(txt_file_path[15:-4]))
    toJson(error_ls, 'error_data_{}.json'.format(txt_file_path[15:-4]))

#중간에 멈췄으면, 아래 코드 실행 후 **반복문 범위와 **파일저장명(file_path) 변경해서 재실행
#print(productUrlList.index(prod_data[-1]))

In [60]:
data_df = pd.read_json('satisfaction_data_과일.json')
data_df

,prod_idx,avg_seller_prod_satis,avg_seller_ship_satis,use_opt1,avg_use_satis1,use_opt2,avg_use_satis2,prod_satis,prod_url
0,210470745,86,90,맛,54,신선도,48,4.3,http://www.tmon.co.kr/deal/210470745\n
1,423178126,88,90,크기차이,54,신선도,52,4.5,http://www.tmon.co.kr/deal/423178126\n
2,403647558,83,81,맛,65,포장,57,4.5,http://www.tmon.co.kr/deal/403647558\n
3,650818926,91,92,맛,78,포장,68,4.5,http://www.tmon.co.kr/deal/650818926\n
4,3730576234,81,84,신선도,59,크기차이,54,4.3,http://www.tmon.co.kr/deal/3730576234\n
